<!-- @format -->

The eventual goal is to update a parquet file with generation data from the French grid.
Once I have made a first step, I will map out a bunch of steps. 


In [1]:
import os
import requests
import functools
import polars as pl

In [2]:
r = requests.post(
    "https://digital.iservices.rte-france.com/token/oauth/",
    headers={
        "Authorization": f'Basic {os.environ["RTE_FRANCE_BASE64"]}',
        "Content-Type": "application/x-www-form-urlencoded",
    },
)

In [3]:
token = r.json()["access_token"]

In [4]:
r_data = requests.get(
    "https://digital.iservices.rte-france.com/open_api/actual_generation/v1/generation_mix_15min_time_scale/",
    headers={
        "Host": "digital.iservices.rte-france.com",
        "Authorization": f"Bearer {token}",
    },
)

In [5]:
array = r_data.json()["generation_mix_15min_time_scale"]

In [6]:
temp = list(
    map(
        lambda x: list(
            map(
                lambda v: {
                    "type": x["production_type"],
                    "interval_start": v["start_date"],
                    "interval_end": v["end_date"],
                    "generation": v["value"],
                },
                x["values"],
            )
        ),
        array,
    )
)

In [7]:
fixed = functools.reduce(lambda x, y: list(x, y), temp)

In [8]:
df = pl.DataFrame(fixed)

In [9]:
df.select(
    [
        pl.col("type"),
        pl.col("interval_start")
        .str.strptime(pl.Datetime("ms"))
        .dt.convert_time_zone(time_zone="Europe/Paris"),
        pl.col("interval_end")
        .str.strptime(pl.Datetime("ms"))
        .dt.convert_time_zone(time_zone="Europe/Paris"),
        pl.col("generation"),
    ]
)

type,interval_start,interval_end,generation
str,"datetime[ms, Europe/Paris]","datetime[ms, Europe/Paris]",i64
"""BIOENERGY""",2023-08-17 00:00:00 CEST,2023-08-17 00:15:00 CEST,0
"""BIOENERGY""",2023-08-17 00:15:00 CEST,2023-08-17 00:30:00 CEST,0
"""BIOENERGY""",2023-08-17 00:30:00 CEST,2023-08-17 00:45:00 CEST,0
"""BIOENERGY""",2023-08-17 00:45:00 CEST,2023-08-17 01:00:00 CEST,0
"""BIOENERGY""",2023-08-17 01:00:00 CEST,2023-08-17 01:15:00 CEST,0
"""BIOENERGY""",2023-08-17 01:15:00 CEST,2023-08-17 01:30:00 CEST,0
"""BIOENERGY""",2023-08-17 01:30:00 CEST,2023-08-17 01:45:00 CEST,0
"""BIOENERGY""",2023-08-17 01:45:00 CEST,2023-08-17 02:00:00 CEST,0
"""BIOENERGY""",2023-08-17 02:00:00 CEST,2023-08-17 02:15:00 CEST,0
